In [2]:
from bs4 import BeautifulSoup
import requests
from urllib.request import urlopen, urlretrieve
from selenium import webdriver
from tqdm import tqdm
import os, re, time, random
from multiprocessing.dummy import Pool as ThreadPool
import multiprocessing
import pandas as pd
from functools import reduce
import inspect
from selenium.webdriver.common.keys import Keys

# Analyze Pandas usage in Kaggle Kernels

## 1. Get urls of top kernels by vote

In [ ]:
browser = webdriver.Chrome(os.getcwd() + '/dependencies/chromedriver')

browser.get("https://www.kaggle.com/kernels?sortBy=votes&group=everyone&pageSize=20&language=Python")
time.sleep(1)

elem = browser.find_element_by_tag_name("body")

no_of_pagedowns = 150

while no_of_pagedowns:
    elem.send_keys(Keys.PAGE_DOWN)
    time.sleep(1)
    no_of_pagedowns-=1

soup = BeautifulSoup(browser.page_source, "html.parser")

In [ ]:
kernel_links = ['https://www.kaggle.com' + a['href'] + '/code' for a in soup.find_all("a", class_="block-link__anchor") if ('teaching-notebook-for-total-imaging-newbies' not in a['href'] and 'mentions-of-kaggle-on-hacker-news' not in a['href'])]

In [ ]:
len(kernel_links)

## 2. Get url of code files from kernels page

In [ ]:
browser = webdriver.Chrome('/Users/adithya/Documents/university/research/scrapingKaggle/chromedriver')

In [ ]:
def get_code_from_kernel(kernel_link):
    if ('stkbailey' in kernel_link or 
        'mentions-of-kaggle-on-hacker-news' in kernel_link or 
        'keras-rcnn-based-overview-wip' in kernel_link or 
        'why-employees-leave-and-how-to-predict-it' in kernel_link or 
        'boosting-stacking-and-bayes-searching' in kernel_link or
        'resnet50-features-xgboost' in kernel_link or 
        'github-commit-messages' in kernel_link):
        return None
    try:
        browser.get(kernel_link)
        innerHTML = browser.execute_script("return document.body.innerHTML")
        soup = BeautifulSoup(innerHTML, 'html.parser')

        link = soup.find_all("a", class_="script-code-pane__download")[0]['href']

        return link
    except Exception as e:
        print("E", e)
        return None

In [ ]:
def pull_partition(start, end, partition):
    code_links_write_part = []
    for kl in tqdm(kernel_links[start:end]):
        code_links_write_part.append(get_code_from_kernel(kl))
        time.sleep(1)

    code_file_write_part = open('links/code_links_part' + str(partition) + '.txt', 'w')
    for code_link in code_links_write_part:
        code_file_write_part.write("%s\n" % code_link)
        
    print("Finished partition %d" % partition)

In [ ]:
STEP = 100
partitions = [(STEP*i, STEP*(i+1), i) for i in range(len(kernel_links)//STEP + 1)]
for p in partitions[17:]:
    pull_partition(p[0], p[1], p[2])

In [3]:
code_links_read = []
for i in range(19):
    with open('links/code_links_part' + str(i) + '.txt', 'r') as code_file_read:
        code_links_read += code_file_read.read().splitlines()

In [5]:
code_links = list(set(['https://www.kaggle.com' + cl for cl in code_links_read]))
code_links[:5]

['https://www.kaggle.com/kernels/scriptcontent/323105/download',
 'https://www.kaggle.com/kernels/scriptcontent/3251019/download',
 'https://www.kaggle.com/kernels/scriptcontent/1028149/download',
 'https://www.kaggle.com/kernels/scriptcontent/1292737/download',
 'https://www.kaggle.com/kernels/scriptcontent/1447364/download']

## 3. Pull raw code file to local dir

In [6]:
for code_link in tqdm(code_links):
    !wget --content-disposition $code_link -P data

  0%|          | 0/1845 [00:00<?, ?it/s]

--2018-04-18 20:53:59--  https://www.kaggle.com/kernels/scriptcontent/323105/download
Resolving www.kaggle.com (www.kaggle.com)... 168.62.224.124
Connecting to www.kaggle.com (www.kaggle.com)|168.62.224.124|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2663 (2.6K) [application/octet-stream]
Saving to: ‘data/script.py’

script.py           100%[===================>]   2.60K  --.-KB/s    in 0s      

2018-04-18 20:53:59 (16.3 MB/s) - ‘data/script.py’ saved [2663/2663]



  0%|          | 1/1845 [00:00<15:13,  2.02it/s]

--2018-04-18 20:53:59--  https://www.kaggle.com/kernels/scriptcontent/3251019/download
Resolving www.kaggle.com (www.kaggle.com)... 168.62.224.124
Connecting to www.kaggle.com (www.kaggle.com)|168.62.224.124|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12848 (13K) [application/octet-stream]
Saving to: ‘data/script.ipynb’

script.ipynb        100%[===================>]  12.55K  --.-KB/s    in 0.001s  

2018-04-18 20:53:59 (22.4 MB/s) - ‘data/script.ipynb’ saved [12848/12848]



  0%|          | 2/1845 [00:00<15:13,  2.02it/s]

...

100%|█████████▉| 1843/1845 [18:58<00:01,  1.62it/s]

--2018-04-18 21:12:57--  https://www.kaggle.com/kernels/scriptcontent/1800232/download
Resolving www.kaggle.com (www.kaggle.com)... 168.62.224.124
Connecting to www.kaggle.com (www.kaggle.com)|168.62.224.124|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 102443 (100K) [application/octet-stream]
Saving to: ‘data/script.ipynb.1265’

script.ipynb.1265   100%[===================>] 100.04K   503KB/s    in 0.2s    

2018-04-18 21:12:58 (503 KB/s) - ‘data/script.ipynb.1265’ saved [102443/102443]



100%|█████████▉| 1844/1845 [18:59<00:00,  1.62it/s]

--2018-04-18 21:12:58--  https://www.kaggle.com/kernels/scriptcontent/1123946/download
Resolving www.kaggle.com (www.kaggle.com)... 168.62.224.124
Connecting to www.kaggle.com (www.kaggle.com)|168.62.224.124|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2209 (2.2K) [application/octet-stream]
Saving to: ‘data/script.py.497’

script.py.497       100%[===================>]   2.16K  --.-KB/s    in 0s      

2018-04-18 21:12:58 (54.0 MB/s) - ‘data/script.py.497’ saved [2209/2209]



100%|██████████| 1845/1845 [18:59<00:00,  1.62it/s]


## 4. Clean non-Python files from downloaded

In [7]:
for filename in os.listdir("data"):
    if filename == ".DS_Store":
        pass
    
    name_parts = filename.split(".")
    if len(name_parts) == 2:
        name_parts.append("0")
        
    if name_parts[1] in ['ipynb', 'py']:
        new_name = name_parts[0] + name_parts[2] + "." + name_parts[1]
        new_name = 'r' + new_name if name_parts[1] == 'py' else new_name
        os.rename('data/' + filename, 'data/' + new_name)

    else:
        os.remove('data/' + filename)

## Tokenize Notebooks and Scripts

In [8]:
# Analyze python script
pd_search_tokens = ["." + p[0] for p in inspect.getmembers(pd) if p[1].__class__.__name__ == 'function'] + ['.DataFrame']
df_search_tokens = ["." + p[0] for p in inspect.getmembers(pd.DataFrame) if p[1].__class__.__name__ == 'function']

def parse_script(script_name):
    with open(script_name) as script_file:
        code = script_file.read().splitlines()

    ps = {}
    for line in code:
        for token in pd_search_tokens:
            ct = line.count(token)
            if ct > 0 and token not in ps:
                ps['pd' + token] = ct
            elif ct > 0:
                ps['pd' + token] += ct
    
        for token in df_search_tokens:
            ct = line.count(token)
            if ct > 0 and token not in ps:
                ps['df' + token] = ct
            elif ct > 0:
                ps['df' + token] += ct
    
    return ps

In [9]:
all_methods = []
for filename in os.listdir("data"):
    if filename.split(".")[1] == 'py':
        all_methods.append(parse_script('data/' + filename))
    else:
        pfilename = 'data/' + filename
        cfilename = filename.split(".")[0] + '.py'
        if cfilename not in os.listdir("data"):
            !jupyter nbconvert --to=python $pfilename
            time.sleep(1)
        try:
            all_methods.append(parse_script('data/' + cfilename))
        except:
            pass

[NbConvertApp] Converting notebook data/script0.ipynb to python
[NbConvertApp] Writing 5892 bytes to data/script0.py
[NbConvertApp] Converting notebook data/script1.ipynb to python
[NbConvertApp] ERROR | Notebook JSON is invalid: Additional properties are not allowed ('outputs', 'execution_count' were unexpected)

Failed validating 'additionalProperties' in markdown_cell:

On instance['cells'][0]:
{'cell_type': 'markdown',
 'execution_count': None,
 'metadata': {'_cell_guid': '0e51c8b9-9580-4947-989f-b12ec9204a32',
              '_execution_state': 'idle',
              '_uuid': 'df5eb0f8cda0bff2a82d49faf789549a9456bc34',
              'collapsed': False},
 'outputs': ['...0 outputs...'],
 'source': 'In this notebook, I tried to explore various dimensionality '
           'redu...'}
[NbConvertApp] Writing 16146 bytes to data/script1.py
[NbConvertApp] Converting notebook data/script10.ipynb to python
[NbConvertApp] Writing 5095 bytes to data/script10.py
[NbConvertApp] Converting noteboo

[NbConvertApp] Converting notebook data/script1049.ipynb to python
[NbConvertApp] Writing 6976 bytes to data/script1049.py
[NbConvertApp] Converting notebook data/script105.ipynb to python
[NbConvertApp] Writing 46763 bytes to data/script105.py
[NbConvertApp] Converting notebook data/script1050.ipynb to python
[NbConvertApp] Writing 16566 bytes to data/script1050.py
[NbConvertApp] Converting notebook data/script1051.ipynb to python
[NbConvertApp] Writing 6588 bytes to data/script1051.py
[NbConvertApp] Converting notebook data/script1052.ipynb to python
[NbConvertApp] ERROR | Notebook JSON is invalid: Additional properties are not allowed ('execution_count', 'outputs' were unexpected)

Failed validating 'additionalProperties' in markdown_cell:

On instance['cells'][0]:
{'cell_type': 'markdown',
 'execution_count': None,
 'metadata': {'_cell_guid': '5cb12f78-e5a9-47d9-bcfd-ca322744c2ce',
              '_uuid': '2903ba531fb15d3943b80f28774e0d24d0377741'},
 'outputs': ['...0 outputs...'],


[NbConvertApp] Converting notebook data/script109.ipynb to python
[NbConvertApp] Writing 20069 bytes to data/script109.py
[NbConvertApp] Converting notebook data/script1090.ipynb to python
[NbConvertApp] Writing 10330 bytes to data/script1090.py
[NbConvertApp] Converting notebook data/script1091.ipynb to python
[NbConvertApp] Writing 11097 bytes to data/script1091.py
[NbConvertApp] Converting notebook data/script1092.ipynb to python
[NbConvertApp] Writing 23070 bytes to data/script1092.py
[NbConvertApp] Converting notebook data/script1093.ipynb to python
[NbConvertApp] Writing 6165 bytes to data/script1093.py
[NbConvertApp] Converting notebook data/script1094.ipynb to python
[NbConvertApp] Writing 15347 bytes to data/script1094.py
[NbConvertApp] Converting notebook data/script1095.ipynb to python
[NbConvertApp] Writing 12792 bytes to data/script1095.py
[NbConvertApp] Converting notebook data/script1096.ipynb to python
[NbConvertApp] Writing 3394 bytes to data/script1096.py
[NbConvertAp

[NbConvertApp] Writing 11933 bytes to data/script1132.py
[NbConvertApp] Converting notebook data/script1133.ipynb to python
[NbConvertApp] Writing 43117 bytes to data/script1133.py
[NbConvertApp] Converting notebook data/script1134.ipynb to python
[NbConvertApp] Writing 7989 bytes to data/script1134.py
[NbConvertApp] Converting notebook data/script1135.ipynb to python
[NbConvertApp] ERROR | Notebook JSON is invalid: Additional properties are not allowed ('outputs', 'execution_count' were unexpected)

Failed validating 'additionalProperties' in markdown_cell:

On instance['cells'][0]:
{'cell_type': 'markdown',
 'execution_count': None,
 'metadata': {'_cell_guid': '7554f8ad-e926-4cf9-b818-b65ca3e1e767',
              '_uuid': '0c2c1282439b4546d1106e7cc7bb41f9de331318'},
 'outputs': ['...0 outputs...'],
 'source': 'Welcome to the SQL Scavenger Hunt! Each day over the next five '
           'd...'}
[NbConvertApp] Writing 18746 bytes to data/script1135.py
[NbConvertApp] Converting notebook 

[NbConvertApp] Converting notebook data/script1169.ipynb to python
[NbConvertApp] ERROR | Notebook JSON is invalid: Additional properties are not allowed ('execution_count', 'outputs' were unexpected)

Failed validating 'additionalProperties' in markdown_cell:

On instance['cells'][0]:
{'cell_type': 'markdown',
 'execution_count': None,
 'metadata': {'_cell_guid': '78189bf2-57a7-4a69-b65f-c970faf229c2',
              '_uuid': '69cef913e677eccbd8765603e6e91da68358308d'},
 'outputs': ['...0 outputs...'],
 'source': '第一次真正对这么复杂的数据进行操作，有点不知所措。所以下面是参考了其他大佬的笔记，[传送门][1]\n'
           '\n'
           '\n'
           '  [1]: https...'}
[NbConvertApp] Writing 12917 bytes to data/script1169.py
[NbConvertApp] Converting notebook data/script117.ipynb to python
[NbConvertApp] ERROR | Notebook JSON is invalid: Additional properties are not allowed ('execution_count', 'outputs' were unexpected)

Failed validating 'additionalProperties' in markdown_cell:

On instance['cells'][0]:
{'cell_type': 'markdow

...

[NbConvertApp] ERROR | Notebook JSON is invalid: Additional properties are not allowed ('execution_count', 'outputs' were unexpected)

Failed validating 'additionalProperties' in markdown_cell:

On instance['cells'][0]:
{'cell_type': 'markdown',
 'execution_count': None,
 'metadata': {'_cell_guid': 'd1978fcb-da9e-437c-b6c7-137d0c99d55a',
              '_uuid': '9ac48023caa236a18a2150800b7aebcb1d6d6a30'},
 'outputs': ['...0 outputs...'],
 'source': '**Author:** Raoul Malm  \n'
           '\n'
           '**Abstract:**  \n'
           '\n'
           'The dataset consists ...'}
[NbConvertApp] Writing 40681 bytes to data/script840.py
[NbConvertApp] Converting notebook data/script841.ipynb to python
[NbConvertApp] Writing 26987 bytes to data/script841.py
[NbConvertApp] Converting notebook data/script842.ipynb to python
[NbConvertApp] Writing 1213 bytes to data/script842.py
[NbConvertApp] Converting notebook data/script843.ipynb to python
[NbConvertApp] Writing 25199 bytes to data/script843

[NbConvertApp] Converting notebook data/script879.ipynb to python
[NbConvertApp] Writing 8184 bytes to data/script879.py
[NbConvertApp] Converting notebook data/script88.ipynb to python
[NbConvertApp] ERROR | Notebook JSON is invalid: Additional properties are not allowed ('outputs', 'execution_count' were unexpected)

Failed validating 'additionalProperties' in markdown_cell:

On instance['cells'][0]:
{'cell_type': 'markdown',
 'execution_count': None,
 'metadata': {'_cell_guid': '04d4701d-7579-401f-9350-e7b6c368df5f',
              '_execution_state': 'idle',
              '_uuid': '1151a9e323056b0972ed7d8b9d35e0ed176697ab',
              'collapsed': False},
 'outputs': ['...0 outputs...'],
 'source': 'This is the stacking model using NN. This gives the public LB '
           'sc...'}
[NbConvertApp] Writing 15687 bytes to data/script88.py
[NbConvertApp] Converting notebook data/script880.ipynb to python
[NbConvertApp] Writing 22129 bytes to data/script880.py
[NbConvertApp] Convertin

    raise value.with_traceback(tb)
  File "/anaconda3/lib/python3.6/site-packages/nbconvert/exporters/../templates/python.tpl", line 1, in top-level template code
    {%- extends 'null.tpl' -%}
  File "/anaconda3/lib/python3.6/site-packages/nbconvert/exporters/../templates/skeleton/null.tpl", line 26, in top-level template code
    {%- block body -%}
  File "/anaconda3/lib/python3.6/site-packages/nbconvert/exporters/../templates/skeleton/null.tpl", line 28, in block "body"
    {%- block any_cell scoped -%}
  File "/anaconda3/lib/python3.6/site-packages/nbconvert/exporters/../templates/skeleton/null.tpl", line 31, in block "any_cell"
    {%- block codecell scoped -%}
  File "/anaconda3/lib/python3.6/site-packages/nbconvert/exporters/../templates/skeleton/null.tpl", line 33, in block "codecell"
    {%- block input_group -%}
  File "/anaconda3/lib/python3.6/site-packages/nbconvert/exporters/../templates/skeleton/null.tpl", line 37, in block "input_group"
    {%- block input -%}{%- endbloc

[NbConvertApp] Writing 5809 bytes to data/script929.py
[NbConvertApp] Converting notebook data/script93.ipynb to python
[NbConvertApp] Writing 13035 bytes to data/script93.py
[NbConvertApp] Converting notebook data/script930.ipynb to python
[NbConvertApp] Writing 4453 bytes to data/script930.py
[NbConvertApp] Converting notebook data/script931.ipynb to python
[NbConvertApp] Writing 14223 bytes to data/script931.py
[NbConvertApp] Converting notebook data/script932.ipynb to python
[NbConvertApp] Writing 7518 bytes to data/script932.py
[NbConvertApp] Converting notebook data/script933.ipynb to python
[NbConvertApp] Writing 3259 bytes to data/script933.py
[NbConvertApp] Converting notebook data/script934.ipynb to python
[NbConvertApp] Writing 14364 bytes to data/script934.py
[NbConvertApp] Converting notebook data/script935.ipynb to python
[NbConvertApp] Writing 10532 bytes to data/script935.py
[NbConvertApp] Converting notebook data/script936.ipynb to python
[NbConvertApp] Writing 10365 b

[NbConvertApp] Writing 27227 bytes to data/script978.py
[NbConvertApp] Converting notebook data/script979.ipynb to python
[NbConvertApp] ERROR | Notebook JSON is invalid: Additional properties are not allowed ('execution_count', 'outputs' were unexpected)

Failed validating 'additionalProperties' in markdown_cell:

On instance['cells'][0]:
{'cell_type': 'markdown',
 'execution_count': None,
 'metadata': {'_cell_guid': '28ecf92f-5246-4ab8-a741-cbee8ff909d0',
              '_uuid': '18101e64642b5c6d04569c7e7d1239b6c60c780d'},
 'outputs': ['...0 outputs...'],
 'source': '# Table of Contents:\n'
           '1. Introduction\n'
           '2. Imports\n'
           '3. Ratings distr...'}
[NbConvertApp] Writing 27017 bytes to data/script979.py
[NbConvertApp] Converting notebook data/script98.ipynb to python
[NbConvertApp] Writing 29175 bytes to data/script98.py
[NbConvertApp] Converting notebook data/script980.ipynb to python
[NbConvertApp] Writing 8668 bytes to data/script980.py
[NbConvertApp

In [10]:
methods_write = open('results/methods.txt', 'w')
for method in all_methods:
    methods_write.write("%s\n" % method)

In [ ]:
print(len(all_methods))

In [ ]:
base = {}
for dict_method in all_methods:
    for k in dict_method:
        if k not in base:
            base[k] = 0 
        base[k] += dict_method[k]

In [ ]:
m, c = [], []
for method, count in base.items():
    m.append(method)
    c.append(count)
    
mFreq = pd.DataFrame(data = {'Method' : m, 'Count': c}).set_index('Method').sort_values('Count', ascending=False)
mFreq.to_csv('results/results.csv')
mFreq.head(10)